# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [1]:
def exp_return(data, P):
    data['ex_return'] = (data['Price in 1 Year']/P - 1)
    exp_re = sum(data['Probability'] * data['ex_return'])
    # exp_return(data, 40) should output 0.07500000000000002
    return exp_re



Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def volatility(data, P):
    re = exp_return(data, P)
    vol = (sum(data['Probability'] * (data['ex_return'] **2)) - re **2) ** 0.5
    # volatility(data,40) should output 0.21301408404140795
    return vol


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    re = exp_return(data, P)
    vol = volatility(data, P)
    low_int = re - 1.96 * (vol/3 ** 0.5)
    high_int = re + 1.96 * (vol/3 ** 0.5)
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    return [low_int, high_int]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [7]:
def exp_return_capm(beta, market_return, rf):
    capm = rf + beta * (market_return - rf)
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return capm

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [11]:
def clean_data(path):
    df = pd.read_csv(path)
    df = df.dropna()
    df['RET'] = pd.to_numeric(df['RET'], errors = 'coerce')
    df = df.dropna(subset=['RET'])    
    obs_num = df.groupby('PERMNO').size()
    valid = obs_num[obs_num >= 500]
    filtered = df[df['PERMNO'].apply(lambda x: x in valid)]    
    # This should yield data with 403316 rows × 4 columns
    return filtered 


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [15]:
def security_volatilities(data):
    output = data.groupby('PERMNO')['RET'].std()
    output = output.reset_index()
    output = output.rename(columns = {'PERMNO': 'CUSIP', 'RET': 'Volatility'})    
    # This should yield data with 679 rows × 2 columns
    return output


In [12]:
def avg_return(data):
    avg_re = data.groupby('PERMNO')['RET'].mean()
    # This should yield data with 679 rows × 2 columns
    output = avg_re.reset_index()
    output = output.rename(columns ={'PERMNO': 'CUSIP', 'RET': 'Average Return'})   
    return output



Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [17]:
def correlation(ret_data, vol_data):
    corr = np.corrcoef(ret_data['Average Return'], vol_data['Volatility'])
    # correlation(rets, vols) should yield 0.44837090727059614
    return corr

